# KDD cup NN development

#### Alex Liu

This is a development version to test NN on KDD 2017

Notes

    1. Currently using only data after 9/19
    2. Currently ignoring those misssing features
    3. Validation 10/11 ~ 10/17
    4. Find suitable epoch or Try early terminaltion
    5. Go deeper? (Not enough entropy, try 3 layer)
    6. Dropout & L2 reg
    7. Find some ways to initailize weights better
    8. Try CNN
    9. Task dependent Model
    10.Missing value filling
    11.Involve weather information?
    12.Involve long vacation?
    13.All other infos!

## Parameters Record

    85->85->50->11, None/Relu/Linear, seed 123, batch size 8, epoch 500
    
    Volume MAPE :  0.163315335319
    
    Traffic time MAPE :  0.272088013453
    
    Volume MAPE :  0.0882439300927


## Library & Parameter Settings

In [50]:
from src import functions as func
import datetime
import numpy as np
from datetime import datetime,timedelta
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from keras.wrappers.scikit_learn import KerasRegressor


seed = 123
np.random.seed(seed)

feature_dim = 85
label_dim = 11 
instance_num = (22)*12  #total 13 days, 12 time windows per day, one week for validation

## Training Data Source

You can skip this part.

Task 1 : traffic time

Task 2 : volumes


In [102]:
file_suffix = '.csv'
path = '../dataSets/training/'  # set the data directory
in_file = 'volume(table 6)_training'
volumes = func.read_file_to_volumes(path, in_file, file_suffix)

in_file = 'trajectories(table 5)_training'
travel_times = func.read_file_to_travel_times(path, in_file, file_suffix)

weathers = func.read_weather(path, 'weather (table 7)_training', file_suffix)
links = func.read_links(path, 'links (table 3)', file_suffix)
routes = func.read_routes(path, 'routes (table 4)', file_suffix)



In [103]:

all_label = {}
for route in travel_times.keys():
    route_time_windows = list(travel_times[route].keys())
    route_time_windows.sort()
    for time_window_start in route_time_windows:
        if time_window_start not in all_label.keys():
            all_label[time_window_start] = {}
        if 'avg_time' not in all_label[time_window_start].keys():
            all_label[time_window_start]['avg_time'] = {}
        tt_set = travel_times[route][time_window_start]
        avg_tt = round(sum(tt_set) / float(len(tt_set)), 2)
        all_label[time_window_start]['avg_time'][route] = avg_tt

time_windows = list(volumes.keys())
time_windows.sort()
for time_window_start in time_windows:
    if time_window_start not in all_label.keys():
        all_label[time_window_start] = {}
    if 'volume' not in all_label[time_window_start].keys():
        all_label[time_window_start]['volume'] = {}
    for tollgate_id in volumes[time_window_start]:
        if tollgate_id not in all_label[time_window_start].keys():
            all_label[time_window_start]['volume'][tollgate_id] = {}
        for direction in volumes[time_window_start][tollgate_id]:
            all_label[time_window_start]['volume'][tollgate_id][direction] = volumes[time_window_start][tollgate_id][direction]




## Feature Engineering

Each instance is a day.

Features:

0~6   

        Monday ~ Sunday (one-hot)

7~18
        
        which window own this instance's label (one-hot)

19~48 

        6:00~8:00 tollgate volume (6 windows * 5 tollgate&dir)
      
        15:00~17:00 tollgate volume (6 windows * 5 tollgate&dir)


49~84

        6:00~8:00 avg. travel time (6 windows * 6 route travel time)

        15:00~17:00 avg. travel time (6 windows * 6 route travel time)

85~90

        label of avg. travel time prediction

91~95

        label of volume prediction




In [104]:
#given a date and time, return volume/traffic_time for next <num_window> windows
def get_source(all_label,date,time,num_window,target):
    all_tol = ['1','3','2']
    all_dir = ['0','1']
    all_comb = ['A-2','A-3','B-1','B-3','C-1','C-3']
    source = np.zeros((1,num_window*5)) if target == 'volume' else np.zeros((1,num_window*6))
    target_window = datetime.strptime(date+' '+time, "%Y-%m-%d %H:%M:%S")
    window_size = timedelta(minutes=20)
    
    for i in range(num_window):
        if target == 'volume':
            for j in range(len(all_tol)):
                for k in range(len(all_dir)):
                    if all_tol[j] == '2' and all_dir[k] == '1':
                        continue
                    try:
                        source[0,i*5+j*2+k] = all_label[target_window][target][all_tol[j]][all_dir[k]]
                    except:
                        source[0,i*5+j*2+k] = 0
        else:
            for j in range(len(all_comb)):
                try:
                    source[0,i*6+j] = all_label[target_window][target][all_comb[j]]
                except:
                    source[0,i*6+j] = 0
        target_window += window_size  
    return source

def my_mape(pred,label):
    vol_mape = 0.0
    traf_mape = 0.0
    
    traf_p = pred[:,:6]
    vol_p = pred[:,6:]
    traf_y = label[:,:6]
    vol_y = label[:,6:]
    c_vol = 0
    c_traf = 0
    
    for (lp,ly) in zip(traf_p,traf_y):
        for (p,y) in zip(lp,ly):
            if y != 0:
                traf_mape += np.abs((p-y)/y)
                c_traf += 1
    
    for (lp,ly) in zip(vol_p,vol_y):
        for (p,y) in zip(lp,ly):
            if y != 0:
                vol_mape += np.abs((p-y)/y)
                c_vol +=1
    
    vol_mape /= c_vol
    traf_mape /= c_traf
        
    return vol_mape,traf_mape

In [105]:
avg_time_null = 0
volume_null = 0

Data = np.zeros((instance_num,feature_dim+label_dim))
num_avg_time = 6
num_volume = 5
all_comb = ['A-2','A-3','B-1','B-3','C-1','C-3']
all_tol = ['1','3','2']
all_dir = ['0','1']
current_weekday = 0 #It's Monday on 2016/9/19
current_pred_window = 0 #8:00~10:00,17:00~19:00
target_window = '2016-9-19 08:00:00'
target_window = datetime.strptime(target_window, "%Y-%m-%d %H:%M:%S")
source_window = target_window.date()

for i in range(instance_num):
    # Weekday (One hot encoding)
    Data[i][current_weekday] = 1
    
    # Predict target window (One hot encoding)
    Data[i][7+current_pred_window] = 1
    
    #Source timewindow volume 6:00~8:00/15:00~17:00
    if(current_pred_window)<6:
        Data[i,19:49] = get_source(all_label,str(target_window.date()),'06:00:00',6,'volume')
    else:
        Data[i,19:49] = get_source(all_label,str(target_window.date()),'15:00:00',6,'volume')

    #Source timewindow avg traffic time 6:00~8:00/15:00~17:00
    if(current_pred_window)<6:
        Data[i,49:85]= get_source(all_label,str(target_window.date()),'06:00:00',6,'avg_time')
    else:
        Data[i,49:85]= get_source(all_label,str(target_window.date()),'15:00:00',6,'avg_time')

    #Label
    Data[i,feature_dim:feature_dim+num_avg_time] = get_source(all_label,str(target_window.date()),str(target_window.time()),1,'avg_time')
    Data[i,feature_dim+num_avg_time:] = get_source(all_label,str(target_window.date()),str(target_window.time()),1,'volume')
        
    
    #move to next window
    current_pred_window = (current_pred_window+1)%12
    
    #move to next weekday every 12 windows
    if (i+1) % 12 == 0:
        current_weekday = (current_weekday+1)%7
        
    #compute next window
    if i%12 == 5:
        target_window = target_window + timedelta(minutes=440)
    elif i%12 == 11:
        target_window = target_window + timedelta(minutes=800)
    else:
        target_window = target_window + timedelta(minutes=20)
    if str(target_window.date()) == '2016-10-01':
        target_window = target_window + timedelta(days = 7)


print 'Total instance : ',instance_num


Total instance :  84


In [106]:
#Cut data into training and validation set
#Last 7 days as validation set (10/11~10/17)
trX = Data[:instance_num-7,:feature_dim]
trY = Data[:instance_num-7,feature_dim:]
valX = Data[instance_num-7:,:feature_dim]
valY = Data[instance_num-7:,feature_dim:]



# Model


In [ ]:
# Create model, modify model here (if you want)
# Here, it's a 4 layer NN with dimension as follow
# [85] -> [85] -> [50] -> [11]
act = 'linear'
model = Sequential()
model.add(Dense(feature_dim, input_dim=feature_dim, init='normal', activation=None))
model.add(Dense(int(feature_dim/2), input_dim=feature_dim, init='normal', activation='relu'))
model.add(Dense(label_dim, init='normal', activation='linear'))

# Compile model
model.compile(loss='mae', optimizer='adam')

# Fit the model
model.fit(trX,trY, nb_epoch=500, batch_size=8,verbose=1)

# evaluate the model
scores = model.evaluate(valX, valY)



In [ ]:
prediction = model.predict(valX)
vol_mape,traf_mape = my_mape(prediction,valY)
print 'Volume MAPE : ',vol_mape
print 'Traffic time MAPE : ',traf_mape
print prediction
print valY


In [116]:
#trX = Data[:,:feature_dim]
#trY = Data[:,feature_dim:]
#valX = Data[instance_num-7:,:feature_dim]
#valY = Data[instance_num-7:,feature_dim:]

vol_trY = trY[:,6:]
vol_valY = valY[:,6:]

model = Sequential()
model.add(Dense(40, input_dim=feature_dim, init='normal', activation='relu'))
model.add(Dense(5, init='normal', activation='linear'))

# Compile model
model.compile(loss='mae', optimizer='adam')

# Fit the model
model.fit(trX,vol_trY, nb_epoch=500, batch_size=8,verbose=1)

# evaluate the model
scores = model.evaluate(valX, vol_valY)


Epoch 1/500
77/77 [==============================] - 0s - loss: 84.2665     
Epoch 2/500
77/77 [==============================] - 0s - loss: 54.7841     
Epoch 3/500
77/77 [==============================] - 0s - loss: 32.5913     
Epoch 4/500
77/77 [==============================] - 0s - loss: 29.7815     
Epoch 5/500
77/77 [==============================] - 0s - loss: 25.4687     
Epoch 6/500
77/77 [==============================] - 0s - loss: 24.5981     
Epoch 7/500
77/77 [==============================] - 0s - loss: 22.2142     
Epoch 8/500
77/77 [==============================] - 0s - loss: 20.8110     
Epoch 9/500
77/77 [==============================] - 0s - loss: 19.5717     
Epoch 10/500
77/77 [==============================] - 0s - loss: 18.6219     
Epoch 11/500
77/77 [==============================] - 0s - loss: 17.4918     
Epoch 12/500
77/77 [==============================] - 0s - loss: 16.8680     
Epoch 13/500
77/77 [==============================] - 0s - loss: 16.1399 

In [117]:
print trX.shape
print np.mean(np.abs(model.predict(valX).flatten()-vol_valY.flatten())/vol_valY.flatten())

(77, 85)
0.300999980977


# Testing Prediction

In [67]:
file_suffix = '.csv'
path = '../dataSets/testing_phase1/'  # set the data directory
in_file = 'volume(table 6)_test1'
volumes = func.read_file_to_volumes(path, in_file, file_suffix)

in_file = 'trajectories(table 5)_test1'
travel_times = func.read_file_to_travel_times(path, in_file, file_suffix)

weathers = func.read_weather(path, 'weather (table 7)_test1', file_suffix)


all_label = {}
for route in travel_times.keys():
    route_time_windows = list(travel_times[route].keys())
    route_time_windows.sort()
    for time_window_start in route_time_windows:
        if time_window_start not in all_label.keys():
            all_label[time_window_start] = {}
        if 'avg_time' not in all_label[time_window_start].keys():
            all_label[time_window_start]['avg_time'] = {}
        tt_set = travel_times[route][time_window_start]
        avg_tt = round(sum(tt_set) / float(len(tt_set)), 2)
        all_label[time_window_start]['avg_time'][route] = avg_tt

time_windows = list(volumes.keys())
time_windows.sort()
for time_window_start in time_windows:
    if time_window_start not in all_label.keys():
        all_label[time_window_start] = {}
    if 'volume' not in all_label[time_window_start].keys():
        all_label[time_window_start]['volume'] = {}
    for tollgate_id in volumes[time_window_start]:
        if tollgate_id not in all_label[time_window_start].keys():
            all_label[time_window_start]['volume'][tollgate_id] = {}
        for direction in volumes[time_window_start][tollgate_id]:
            all_label[time_window_start]['volume'][tollgate_id][direction] = volumes[time_window_start][tollgate_id][direction]





avg_time_null = 0
volume_null = 0
instance_num = 7*12
Data = np.zeros((instance_num,feature_dim+label_dim))
num_avg_time = 6
num_volume = 5
all_comb = ['A-2','A-3','B-1','B-3','C-1','C-3']
all_tol = ['1','3','2']
all_dir = ['0','1']
current_weekday = 1 #It's Tuesday on 2016/9/19
current_pred_window = 0 #8:00~10:00,17:00~19:00
target_window = '2016-10-18 08:00:00'
target_window = datetime.strptime(target_window, "%Y-%m-%d %H:%M:%S")
source_window = target_window.date()

for i in range(instance_num):
    # Weekday (One hot encoding)
    Data[i][current_weekday] = 1
    
    # Predict target window (One hot encoding)
    Data[i][7+current_pred_window] = 1
    
    #Source timewindow volume 6:00~8:00/15:00~17:00
    if(current_pred_window)<6:
        Data[i,19:49] = get_source(all_label,str(target_window.date()),'06:00:00',6,'volume')
    else:
        Data[i,19:49] = get_source(all_label,str(target_window.date()),'15:00:00',6,'volume')

    #Source timewindow avg traffic time 6:00~8:00/15:00~17:00
    if(current_pred_window)<6:
        Data[i,49:85]= get_source(all_label,str(target_window.date()),'06:00:00',6,'avg_time')
    else:
        Data[i,49:85]= get_source(all_label,str(target_window.date()),'15:00:00',6,'avg_time')

    #Label
    Data[i,feature_dim:feature_dim+num_avg_time] = get_source(all_label,str(target_window.date()),str(target_window.time()),1,'avg_time')
    Data[i,feature_dim+num_avg_time:] = get_source(all_label,str(target_window.date()),str(target_window.time()),1,'volume')
        
    
    #move to next window
    current_pred_window = (current_pred_window+1)%12
    
    #move to next weekday every 12 windows
    if (i+1) % 12 == 0:
        current_weekday = (current_weekday+1)%7
        
    #compute next window
    if i%12 == 5:
        target_window = target_window + timedelta(minutes=440)
    elif i%12 == 11:
        target_window = target_window + timedelta(minutes=800)
    else:
        target_window = target_window + timedelta(minutes=20)


In [68]:
ttX = Data[:,:feature_dim]
for i in range(len(ttX)):
    print ttX[i]
ttY = model.predict(ttX)
print ttY.flatten().shape




[   0.      1.      0.      0.      0.      0.      0.      1.      0.
    0.      0.      0.      0.      0.      0.      0.      0.      0.
    0.     13.     37.     30.     23.     24.     17.     47.     42.
   29.     36.     21.     72.     85.     50.     46.     31.     68.
  114.     56.     83.     28.     94.    131.     77.     87.     47.
  105.    164.     91.     97.     41.1    69.48   57.82   54.75  145.35
  139.53   43.68  142.01   48.68  121.02  136.22    0.     68.02  233.68
  112.52   94.04  147.82  104.42   52.61  143.65   97.32  120.6   180.98
  182.06   56.17  151.51    0.     73.4   136.51    0.     63.6   207.71
  129.44   89.91  311.04    0.  ]
[   0.      1.      0.      0.      0.      0.      0.      0.      1.
    0.      0.      0.      0.      0.      0.      0.      0.      0.
    0.     13.     37.     30.     23.     24.     17.     47.     42.
   29.     36.     21.     72.     85.     50.     46.     31.     68.
  114.     56.     83.     28.     

In [69]:
import pandas as pd
tors = ['1','1','3','3','2']
dirs = ['0','1','0','1','0']
test_dates = '2016-10-18 08:00:00'
test_dates = datetime.strptime(test_dates, "%Y-%m-%d %H:%M:%S")
toltal_pred = []
header = ['tollgate_id','time_window','direction','volume']
time_delta = timedelta(minutes=20)
ttY = ttY.flatten()
i = 0 
with open('vol_pred.csv','w') as out:
    for day in range(7):
        for wind in range(12):
            for tor,dire in zip(tors,dirs):
                pred = []
                pred.append(str(tor))
                pred.append('['+str(test_dates)+','+str(test_dates+time_delta)+')')
                pred.append(str(dire))
                pred.append(str(ttY[i]))
                i += 1
                toltal_pred.append(pred)
            #compute next window
            if wind%12 == 5:
                test_dates = test_dates + timedelta(minutes=440)
            elif wind%12 == 11:
                test_dates = test_dates + timedelta(minutes=800)
            else:
                test_dates = test_dates + timedelta(minutes=20)
    df = pd.DataFrame(data=toltal_pred,columns=header)
    df.to_csv(out,index=False,quoting=2)
                